In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
import json
from time import sleep
import nltk
import numpy as np
import argparse
from langchain.llms import OpenAI
import baseline_utils
from dotenv import load_dotenv
from types import SimpleNamespace
import asyncio

In [ ]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List

In [ ]:
class GSM_Answer(BaseModel):
    work: str = Field(description="Explanation of answer")
    final_answer: str = Field(description="Final numeric answer")
    

In [ ]:
parser = PydanticOutputParser(pydantic_object=GSM_Answer)

In [ ]:
load_dotenv()

In [135]:
args = {
    'data_dir': '../../data/gsm_data',
    'save_dir': 'models',
    'debug': False,
    'exp_label': 'default',
    'task': 'pot_gsm',
    'model': 'gpt-3.5-turbo',
    'max_tokens': 2048,
    'temperature': 0.0,
}
args['ckpt_path'] = os.path.join(args['save_dir'], args['exp_label'])
args = SimpleNamespace(**args)

In [ ]:
logger = baseline_utils.Logger(os.path.join(args.ckpt_path, 'log.txt'))
completed_rounds = 0

In [ ]:
async def async_generate_answer(llm, prompt_template, problem):
    inp = prompt_template.format(context = problem['context'])
    # print(inp)
    success = False
    while not success:
      try:
        output = await llm.agenerate([inp])
        print(output)
        success = True
      except Exception as e:
        logger.write(e)
        logger.write(f'API server overloaded. Waiting for 30 seconds...')
        sleep(30)
        continue
    problem['output'] = output.generations[0][0].text
    global completed_rounds
    completed_rounds += 1
    print(f"Completed {completed_rounds} rounds")


In [ ]:
async def async_generate_answers(llm, prompt_template, problems):
  '''Generate the answer for the given problem.'''
  outputs = [async_generate_answer(llm, prompt_template, prob) for prob in problems]
  await asyncio.gather(*outputs)


In [ ]:
async def gsm_run(prompt_template, llm, data):
    global completed_rounds
    completed_rounds = 0
    problems = [{'context': d['input'], 'target': d['target']} for d in data]
    step = 5
    for i in range(0, len(problems), step):
        await async_generate_answers(llm, prompt_template, problems[i:min(i + step, len(problems))])
        print (f"Completed {i + step} problems")
    return problems

In [ ]:
def calc_accuracy(problems):
    return sum([p['correct'] for p in problems]) / len(problems)

In [132]:
def parse_answers(filename, task):
    with open(filename, 'r') as f:
        problems = json.loads(f.read())
    for p in problems:
        p['final_answer'] = baseline_utils.parse_answer(p['output'], task)
        p['correct'] = p['final_answer'] == p['target']
    with open(filename, 'w') as f:
        f.write(json.dumps(problems) + '\n')
    return problems
    

In [133]:
async def gsm_baseline(model, task):
    prompt_template = baseline_utils.create_prompt_template(task)
    llm = OpenAI(
        model_name=model,
        max_tokens=args.max_tokens,
        stop=['\\n\\n', 'A:', 'Q:'],
        temperature=args.temperature,
        openai_api_key = os.getenv('OPEN_AI_API_KEY')
  ) 
    for i in range(3):
        for variant in ['original', 'irc']:
            data = baseline_utils.load_gsm_data(os.path.join(args.data_dir, f'gsmic_mixed_{i}_{variant}.jsonl'))
            if (os.path.exists(os.path.join(args.save_dir, f'gsmic_mixed_{i}_{variant}_output_{model}_{task}.json')) 
                or os.path.exists(os.path.join(args.save_dir, f'hand_gsmic_mixed_{i}_{variant}_output_{model}_{task}.json'))):
                continue
            problems = await gsm_run(prompt_template, llm, data)
            output_file = os.path.join(args.save_dir, f'gsmic_mixed_{i}_{variant}_output_{model}_{task}.json')
            with open(output_file, 'w') as f:
                  f.write(json.dumps(problems) + '\n')
            problems = parse_answers(output_file, task)

            logger.write(f'Accuracy for gsmic_mixed_{i}_{variant} = {calc_accuracy(problems)}')

In [136]:
print(args.model, args.task)
await gsm_baseline(args.model, args.task)

gpt-3.5-turbo pot_gsm
generations=[[Generation(text='Let’s solve the problem by a Python program:\ntotal_students = 120\nstatistics_students = total_students / 2\nseniors_in_statistics = statistics_students * 0.9\nprint(seniors_in_statistics)', generation_info=None)]] llm_output={'token_usage': <OpenAIObject at 0x17519e0f0> JSON: {
  "completion_tokens": 45,
  "prompt_tokens": 142,
  "total_tokens": 187
}, 'model_name': 'gpt-3.5-turbo'}
Completed 1 rounds
generations=[[Generation(text='Let’s solve the problem by a Python program:\ntotal_plants = 2 * 20\nseeds = total_plants * 1\nplanted_seeds = seeds * 0.6\nplanted_trees = planted_seeds / 20\nprint(planted_trees)\nTherefore, James planted 6 trees.', generation_info=None)]] llm_output={'token_usage': <OpenAIObject at 0x17519fad0> JSON: {
  "completion_tokens": 66,
  "prompt_tokens": 134,
  "total_tokens": 200
}, 'model_name': 'gpt-3.5-turbo'}
Completed 2 rounds
generations=[[Generation(text='Let’s solve the problem by a Python program:\

Retrying langchain.llms.openai.acompletion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-UL2nM4hjGA7CzFPYucOtFrPC on tokens per min. Limit: 90000 / min. Current: 88542 / min. Contact us through our help center at help.openai.com if you continue to have issues..
Retrying langchain.llms.openai.acompletion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-UL2nM4hjGA7CzFPYucOtFrPC on tokens per min. Limit: 90000 / min. Current: 88539 / min. Contact us through our help center at help.openai.com if you continue to have issues..
Retrying langchain.llms.openai.acompletion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-UL2nM4hjGA7CzFPYucOtFrPC on tokens per min. Limit: 90000 / min. Curre

generations=[[Generation(text='Let’s solve the problem by a Python program:\nMonday_necklaces = 10\nTuesday_necklaces = 2\nWednesday_bracelets = 5\nWednesday_earrings = 7\nNecklace_beads = 20\nBracelet_beads = 10\nEarring_beads = 5\nTotal_beads = (Monday_necklaces + Tuesday_necklaces) * Necklace_beads + Wednesday_bracelets * Bracelet_beads + Wednesday_earrings * Earring_beads\nprint(Total_beads)', generation_info=None)]] llm_output={'token_usage': <OpenAIObject at 0x17519c650> JSON: {
  "completion_tokens": 124,
  "prompt_tokens": 178,
  "total_tokens": 302
}, 'model_name': 'gpt-3.5-turbo'}
Completed 35 rounds
Completed 35 problems


Retrying langchain.llms.openai.acompletion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-UL2nM4hjGA7CzFPYucOtFrPC on tokens per min. Limit: 90000 / min. Current: 89661 / min. Contact us through our help center at help.openai.com if you continue to have issues..
Retrying langchain.llms.openai.acompletion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-UL2nM4hjGA7CzFPYucOtFrPC on tokens per min. Limit: 90000 / min. Current: 89664 / min. Contact us through our help center at help.openai.com if you continue to have issues..
Retrying langchain.llms.openai.acompletion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-UL2nM4hjGA7CzFPYucOtFrPC on tokens per min. Limit: 90000 / min. Curre

Rate limit reached for default-gpt-3.5-turbo in organization org-UL2nM4hjGA7CzFPYucOtFrPC on tokens per min. Limit: 90000 / min. Current: 89494 / min. Contact us through our help center at help.openai.com if you continue to have issues.
API server overloaded. Waiting for 30 seconds...
Rate limit reached for default-gpt-3.5-turbo in organization org-UL2nM4hjGA7CzFPYucOtFrPC on tokens per min. Limit: 90000 / min. Current: 89485 / min. Contact us through our help center at help.openai.com if you continue to have issues.
API server overloaded. Waiting for 30 seconds...
Rate limit reached for default-gpt-3.5-turbo in organization org-UL2nM4hjGA7CzFPYucOtFrPC on tokens per min. Limit: 90000 / min. Current: 89483 / min. Contact us through our help center at help.openai.com if you continue to have issues.
API server overloaded. Waiting for 30 seconds...
Rate limit reached for default-gpt-3.5-turbo in organization org-UL2nM4hjGA7CzFPYucOtFrPC on tokens per min. Limit: 90000 / min. Current: 8946

...

generations=[[Generation(text='Let’s solve the problem by a Python program:\nLexie_apples = 12\nTom_apples = 2 * Lexie_apples\nTotal_apples = Lexie_apples + Tom_apples\nprint(Total_apples)', generation_info=None)]] llm_output={'token_usage': <OpenAIObject at 0x17519f170> JSON: {
  "completion_tokens": 49,
  "prompt_tokens": 135,
  "total_tokens": 184
}, 'model_name': 'gpt-3.5-turbo'}
Completed 91 rounds


Retrying langchain.llms.openai.acompletion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-UL2nM4hjGA7CzFPYucOtFrPC on tokens per min. Limit: 90000 / min. Current: 88728 / min. Contact us through our help center at help.openai.com if you continue to have issues..


generations=[[Generation(text='Let’s solve the problem by a Python program:\nMarcus_birds = 7\nHumphrey_birds = 11\nDarrel_birds = 9\ntotal_birds = Marcus_birds + Humphrey_birds + Darrel_birds\naverage_birds = total_birds / 3\nprint(average_birds)\nThe average number of birds seen by each person is 9.', generation_info=None)]] llm_output={'token_usage': <OpenAIObject at 0x17519e510> JSON: {
  "completion_tokens": 92,
  "prompt_tokens": 145,
  "total_tokens": 237
}, 'model_name': 'gpt-3.5-turbo'}
Completed 92 rounds
generations=[[Generation(text='Let’s solve the problem by a Python program:\nDaliah_garbage = 17.5\nDewei_garbage = Daliah_garbage - 2\nZane_garbage = 4 * Dewei_garbage\nprint(Zane_garbage)', generation_info=None)]] llm_output={'token_usage': <OpenAIObject at 0x17519c590> JSON: {
  "completion_tokens": 61,
  "prompt_tokens": 157,
  "total_tokens": 218
}, 'model_name': 'gpt-3.5-turbo'}
Completed 93 rounds
generations=[[Generation(text='Let’s solve the problem by a Python prog

Retrying langchain.llms.openai.acompletion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-UL2nM4hjGA7CzFPYucOtFrPC on tokens per min. Limit: 90000 / min. Current: 89507 / min. Contact us through our help center at help.openai.com if you continue to have issues..
Retrying langchain.llms.openai.acompletion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-UL2nM4hjGA7CzFPYucOtFrPC on tokens per min. Limit: 90000 / min. Current: 89511 / min. Contact us through our help center at help.openai.com if you continue to have issues..


generations=[[Generation(text='Let’s solve the problem by a Python program:\nquarters = 160\ndollars_spent = 35 / 0.25\nquarters_left = quarters - dollars_spent\nprint(quarters_left)\n# This will output 6.0, which means Libby will have 6 quarters left after replacing the dress.', generation_info=None)]] llm_output={'token_usage': <OpenAIObject at 0x17519d610> JSON: {
  "completion_tokens": 66,
  "prompt_tokens": 146,
  "total_tokens": 212
}, 'model_name': 'gpt-3.5-turbo'}
Completed 95 rounds
Completed 95 problems
generations=[[Generation(text="Let’s solve the problem by a Python program:\nMonday = 40\nTuesday = 50\nWednesday = Tuesday * 0.5\nThursday = Monday + Wednesday\nTotal_kilometers = Monday + Tuesday + Wednesday + Thursday\nprint(Total_kilometers)\n# Output: 155\nNatalie rode a total of 155 kilometers. The shoe size of Natalie's mother is not relevant to the problem and does not affect the solution.", generation_info=None)]] llm_output={'token_usage': <OpenAIObject at 0x17519fef

In [ ]:
test_question = baseline_utils.load_gsm_data(os.path.join(args.data_dir, f'gsmic_mixed_0_original.jsonl'))[2]

prompt_template = baseline_utils.create_prompt_template('pot_gsm')
llm = OpenAI(
    model_name=args.model, 
    max_tokens=args.max_tokens, 
    stop=['\\n\\n', 'A:', 'Q:'],
    
    temperature=args.temperature,
    openai_api_key = os.getenv('OPEN_AI_API_KEY')
)

In [ ]:
print(prompt_template.format(context = test_question['input']))

In [ ]:

tp = {**test_question, 'context': test_question['input']}
await async_generate_answers(llm, prompt_template, [tp])

In [ ]:
print(baseline_utils.parse_answer(tp['output'], 'pot_gsm'))

In [ ]:
def grade(filepath, rewrite = False):
    with open(filepath, 'r') as f:
        problems = json.load(f)
    for p in problems:
        if (p['target'] == p['final_answer']):
            p['correct'] = True
    if (rewrite):
        filepath = os.path.join('/'.join(filepath.split('/')[:-1]), 'hand_' + filepath.split('/')[-1])
    with open(filepath, 'w') as f:
        f.write(json.dumps(problems) + '\n')
    print(calc_accuracy(problems))


In [ ]:


for task in ['0cot', '1cot_gsm', 'pot_gsm']:
    for i in range(3):
            for variant in ['original', 'irc']:
                filepath = os.path.join(args.save_dir, f'hand_gsmic_mixed_{i}_{variant}_output_{args.model}_{task}.json')
                print(filepath)
                # parse_answers(filepath, args.task)
                print(grade(filepath))
            
